In [ ]:
import numpy as np
import pandas as pd
import time 
from datetime import datetime
from matplotlib import pyplot as plt

data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
data = np.array(data)
m,n = data.shape

np.random.shuffle(data)      # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]          # Label
X_dev = data_dev[1:n]        # Features
X_dev = X_dev / 255.         # Normalization

data_train = data[1000:m].T
Y_train = data_train[0]      # Label
X_train = data_train[1:n]    # Features
X_train = X_train / 255.     # Normalization
_,m_train = X_train.shape


#print(np.size(data[0]))

In [ ]:
def init_params():
    W1 = np.random.rand(64, 784) - 0.5  # First layer (input: 784, hidden: 64)
    b1 = np.random.rand(64, 1) - 0.5
    W2 = np.random.rand(32, 64) - 0.5   # Second layer (hidden: 64, hidden: 32)
    b2 = np.random.rand(32, 1) - 0.5
    W3 = np.random.rand(16, 32) - 0.5   # Third layer (hidden: 32, hidden: 16)
    b3 = np.random.rand(16, 1) - 0.5
    W4 = np.random.rand(10, 16) - 0.5   # Fourth layer (hidden: 16, output: 10)
    b4 = np.random.rand(10, 1) - 0.5
    
    return W1, b1, W2, b2, W3, b3, W4, b4

# ReLU Activation
def ReLU(Z):
    return np.maximum(Z, 0)

# Leaky ReLU Activation
def Leaky_ReLU(Z, alpha=0.01):
    return np.where(Z > 0, Z, alpha * Z)

# Leaky ReLU Derivative
def Leaky_ReLU_deriv(Z, alpha=0.01):
    return np.where(Z > 0, 1, alpha)

# Softmax Function
def softmax(Z):
    A = np.exp(Z) / np.sum(np.exp(Z), axis=0, keepdims=True)
    return A

# Forward Propagation
def forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X):
    Z1 = W1.dot(X) + b1
    A1 = Leaky_ReLU(Z1)  
    Z2 = W2.dot(A1) + b2
    A2 = Leaky_ReLU(Z2)  
    Z3 = W3.dot(A2) + b3
    A3 = Leaky_ReLU(Z3)  
    Z4 = W4.dot(A3) + b4
    A4 = softmax(Z4)
    
    return Z1, A1, Z2, A2, Z3, A3, Z4, A4

# One-hot Encoding
def one_hot(Y):
    one_hot_Y = np.zeros((Y.max() + 1, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

# Backpropagation
def backward_prop(Z1, A1, Z2, A2, Z3, A3, Z4, A4, W1, W2, W3, W4, X, Y):
    m = X.shape[1]
    one_hot_Y = one_hot(Y)
    
    dZ4 = A4 - one_hot_Y
    dW4 = 1 / m * dZ4.dot(A3.T)
    db4 = 1 / m * np.sum(dZ4, axis=1, keepdims=True)

    dZ3 = W4.T.dot(dZ4) * Leaky_ReLU_deriv(Z3)  
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3, axis=1, keepdims=True)

    dZ2 = W3.T.dot(dZ3) * Leaky_ReLU_deriv(Z2)  
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * Leaky_ReLU_deriv(Z1)  
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3, dW4, db4

# Update Parameters
def update_params(W1, b1, W2, b2, W3, b3, W4, b4, dW1, db1, dW2, db2, dW3, db3, dW4, db4, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2  
    W3 = W3 - alpha * dW3  
    b3 = b3 - alpha * db3  
    W4 = W4 - alpha * dW4  
    b4 = b4 - alpha * db4  
    
    return W1, b1, W2, b2, W3, b3, W4, b4


In [ ]:
def get_predictions(A4):
    return np.argmax(A4, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3, W4, b4 = init_params()
    Start_time = time.time()
    print("Start Time:", datetime.fromtimestamp(Start_time))
    for i in range(iterations+1):
        Z1, A1, Z2, A2, Z3, A3, Z4, A4 = forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X)
        dW1, db1, dW2, db2, dW3, db3, dW4, db4 = backward_prop(Z1, A1, Z2, A2, Z3, A3, Z4, A4, W1, W2, W3, W4, X, Y)
        W1, b1, W2, b2, W3, b3, W4, b4 = update_params(W1, b1, W2, b2, W3, b3, W4, b4, dW1, db1, dW2, db2,dW3, db3, dW4, db4, alpha)
        if i % 1000 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A4)
            print(get_accuracy(predictions, Y))

    End_time = time.time()
    print("End Time:", datetime.fromtimestamp(End_time))
    return W1, b1, W2, b2, W3, b3, W4, b4



In [ ]:
W1, b1, W2, b2, W3, b3, W4, b4 = gradient_descent(X_train, Y_train, 0.20, 20000)

In [ ]:
def make_predictions(X, W1, b1, W2, b2, W3, b3, W4, b4):
    _, _, _, _, _, _, _, A4 = forward_prop(W1, b1, W2, b2, W3, b3, W4, b4, X)
    predictions = get_predictions(A4)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3, W4, b4):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2, W3, b3, W4, b4)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(667, W1, b1, W2, b2, W3, b3, W4, b4)
test_prediction(661, W1, b1, W2, b2, W3, b3, W4, b4)
test_prediction(58, W1, b1, W2, b2, W3, b3, W4, b4)